In [1]:
import numpy as np
from scipy.stats import beta
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest

Frequentist method:


Визначення параметрів тесту: Задано відсоток придбань для контрольної групи (p_control = 0.17) та тестової групи (p_test = 0.20). Також вказані рівень значущості (alpha = 0.05) та статистична потужність (power = 0.80).

Розрахунок ефекту: Обчислюється ефект розміру (effect_size) між контрольними та тестовими групами.

Розрахунок необхідного розміру вибірки: Використовується розмір вибірки для нормального незалежного тесту, з урахуванням ефекту, потужності та рівня значущості.

Оцінка тривалості тесту: Визначається кількість користувачів, які щодня бачать пропозицію (daily_users_seeing_offer), та розраховується тривалість тесту в днях на основі загального розміру вибірки.

Виведення результатів: Показується необхідний розмір вибірки на кожну групу та оцінка тривалості тесту для кожної групи і загалом.

In [93]:
p_control = 0.17
p_test = 0.20  

alpha = 0.05
power = 0.80

effect_size = sms.proportion_effectsize(p_control, p_test)


sample_size = sms.NormalIndPower().solve_power(effect_size, power=power, alpha=alpha, ratio=1)
sample_size = int(sample_size)


daily_users = 2000
daily_users_seeing_offer = int(daily_users * 0.13)
days = int(sample_size / daily_users_seeing_offer)

print(f"Required sample size per group: {sample_size}")
print(f"Estimated test duration (per group): {days} days")

Required sample size per group: 2625
Estimated test duration (per group): 10 days


Визначення даних: Встановлюються відсотки придбань для контрольної (p_control) і тестової (p_test) груп. Генеруються дані про придбання для кожної групи за допомогою біноміального розподілу, де кожен користувач або здійснює покупку (1) або не здійснює (0).

Обчислення статистик: Підраховуються загальна кількість придбань (x_control та x_test) для контрольної і тестової груп. Обчислюється Z-статистика та p-значення за допомогою Z-тесту для пропорцій.

В коді використовується нормальна апроксимація біноміального розподілу, щоб спростити обчислення Z-статистики. Це можливе завдяки великій кількості спроб (показники n), при яких біноміальний розподіл наближається до нормального.

Умови для використання:

Очікувана кількість успіхів (np) та невдач (n(1-p)) повинні бути не меншими за 5.

Результати тесту: Виводяться Z-статистика і p-значення. Перевіряється, чи є p-значення меншим за рівень значущості (alpha). Якщо так, гіпотеза про відсутність різниці відхиляється.

In [91]:
np.random.seed(42)
p_control = 0.17
p_test = 0.20
alpha = 0.05

control_conversions = np.random.binomial(1, p_control, daily_users_seeing_offer * days)
test_conversions = np.random.binomial(1, p_test, daily_users_seeing_offer * days)

n_control = daily_users_seeing_offer * days
x_control = np.sum(control_conversions)
n_test = daily_users_seeing_offer * days
x_test = np.sum(test_conversions)

count = np.array([x_control, x_test])
nobs = np.array([n_control, n_test])
stat, p_value = proportions_ztest(count, nobs)

print(f"Z-statistic: {stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in conversion rates.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in conversion rates.")

def check_normal_approximation(n, p):
    np_value = n * p
    n_p_value = n * (1 - p)
    return np_value > 5 and n_p_value > 5

check_normal_approximation_control = check_normal_approximation(daily_users_seeing_offer, p_control)
check_normal_approximation_test = check_normal_approximation(daily_users_seeing_offer, p_test)

print(f"Control group normal approximation valid: {check_normal_approximation_control}")
print(f"Test group normal approximation valid: {check_normal_approximation_test}")

if check_normal_approximation_control and check_normal_approximation_test:
    print("Normal approximation is valid for both groups.")
else:
    print("Normal approximation is not valid for one or both groups.")


Z-statistic: -2.5163
P-value: 0.0119
Reject the null hypothesis: There is a significant difference in conversion rates.
Control group normal approximation valid: True
Test group normal approximation valid: True
Normal approximation is valid for both groups.


Baysian method:

Визначення апріорних розподілів: Встановлюються апріорні параметри для розподілу Бета (alpha_prior = 1, beta_prior = 1).

Обчислення постеріорних розподілів: Розраховуються постеріорні розподіли для контрольної (posterior_control) і тестової (posterior_test) груп на основі кількості придбань (x_control і x_test) та загальної кількості користувачів (n_control і n_test).

Генерація вибірки: Створюються вибірки з постеріорних розподілів для кожної групи (по 10,000 зразків).

Оцінка ймовірності: Визначається ймовірність того, що конверсія в тестовій групі перевищує конверсію в контрольній групі.

Результати: Виводяться ймовірність перевищення конверсії тестової групи над контрольною, а також середні значення конверсій для обох груп.

In [92]:
alpha_prior = 1
beta_prior = 1

posterior_control = beta(alpha_prior + x_control, beta_prior + n_control - x_control)
posterior_test = beta(alpha_prior + x_test, beta_prior + n_test - x_test)

samples_control = posterior_control.rvs(10000)
samples_test = posterior_test.rvs(10000)

prob_test_beats_control = np.mean(samples_test > samples_control)

mean_control = posterior_control.mean()
mean_test = posterior_test.mean()

print(f"Probability that test group conversion rate is higher than control group: {prob_test_beats_control:.2f}")
print(f"Control group conversion rate: {mean_control:.2%}")
print(f"Test group conversion rate: {mean_test:.2%}")

Probability that test group conversion rate is higher than control group: 0.99
Control group conversion rate: 16.87%
Test group conversion rate: 19.56%
